In [1]:
using JuMP
using Ipopt
using Complementarity

In [2]:
GU = Dict(:pi => 0.01, :L => 0.5, :gamma => 0.02, :sigma => 0.5)

GU[:rho] = 1 - 1/GU[:sigma]

-1.0

In [3]:
function insurance(GU)

    pi = GU[:pi]
    L = GU[:L]
    γ = GU[:gamma]
    σ = GU[:sigma]
    ρ = GU[:rho]

    m = Model(Ipopt.Optimizer)

    @variables(m,begin
        C_G, (start=1,)
        C_B, (start=1,)
        K, (start=1,)
    end)

    @NLobjective(m,Max,(1-pi)* C_G^ρ/ρ+ pi*C_B^ρ/ρ)

    @NLconstraints(m,begin
        budget_G, C_G == 1-γ*K
        budget_B, C_B == 1-L + (1-γ)*K
    end)


    return m
end



function insurance_ev(GU)
    pi = GU[:pi]
    L = GU[:L]
    γ = GU[:gamma]
    σ = GU[:sigma]
    ρ = GU[:rho]

    m = Model(Ipopt.Optimizer)

    @variables(m,begin
        C_G, (start=1,)
        C_B, (start=1,)
        K, (start=1,)
    end)

    @NLobjective(m,Max,100* (( (1-pi)*C_G^ρ + pi * C_B^ρ)^(1/ρ) - 1))

    @NLconstraints(m,begin
        budget_G, C_G == 1-γ*K
        budget_B, C_B == 1-L + (1-γ)*K
    end)


    return m

end

function equilibrium(GU)
    pi = GU[:pi]
    L = GU[:L]
    γ = GU[:gamma]
    σ = GU[:sigma]
    ρ = GU[:rho]

    m = MCPModel()

    @variables(m,begin
        EU, (start=1,)
        EV, (start=1,)
        C_G, (start=1,)
        C_B, (start=1,)
        K, (start=1,)
    end)

    @mapping(m,eudef,EU - ((1-pi)* C_G^(ρ)/ρ+ pi*C_B^(ρ)/ρ))

    @mapping(m,evdef, EV - 100* (( (1-pi)*C_G^ρ + pi * C_B^ρ)^(1/ρ) - 1))

    @mapping(m,budget_G, C_G - (1-γ*K))

    @mapping(m,budget_B, C_B - (1-L + (1-γ)*K))

    @mapping(m,coverage, γ*((1-pi)*C_G^(ρ-1) + pi*C_B^(ρ-1)) - pi*C_B^(ρ-1))
    
    @complementarity(m,eudef,EU)
    @complementarity(m,evdef,EV)
    @complementarity(m,budget_G,C_G)
    @complementarity(m,budget_B,C_B)
    @complementarity(m,coverage,K)

    return m
end

equilibrium (generic function with 1 method)

In [4]:
I = insurance(GU)
set_silent(I)
optimize!(I)

println("Objective -> $(objective_value(I))")




******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Objective -> -1.0083430790292782


In [5]:
I_ev = insurance_ev(GU)
set_silent(I_ev)
optimize!(I_ev)

println("Objective -> $(objective_value(I_ev))")

Objective -> -0.8274047992980704


In [6]:
c = equilibrium(GU)
set_silent(c)

solveMCP(c);



Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.2871e+00             0.0e+00 (eudef)
    1     1     0     5 5.2034e-01  1.0e+00    0.0e+00 (evdef)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 5.2034e-01           I 0.0e+00 5.2e-01 (evdef)
    1     1     3     3 1.0494e-01  1.0e+00 SO 0.0e+00 1.0e-01 (evdef)
    2     1     4     4 1.8864e-02  1.0e+00 SO 0.0e+00 1.9e-02 (evdef)
    3     1     5     5 6.1192e-04  1.0e+00 SO 0.0e+00 6.1e-04 (evdef)
    4     1     6     6 5.9024e-07  1.0e+00 SO 0.0e+00 5.9e-07 (evdef)

Major Iterations. . . . 4
Minor Iterations. . . . 4
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 6
Gradient Evaluations. . 6
Basis Time. . . . . 

In [9]:
println("Var -> I      -> I_ev   -> Equi ")
for var in [:C_G,:C_B,:K]
    i_v = round(value(I[var]),digits=4)
    ie_v = round(value(I_ev[var]),digits=4)
    c_v = round(result_value(c[var]),digits=4)
    println("$var -> $(i_v) -> $(ie_v) -> $(c_v)")
end

Var -> I      -> I_ev   -> Equi 
C_G -> 0.9959 -> 0.9959 -> 0.9959
C_B -> 0.7006 -> 0.7006 -> 0.7006
K -> 0.2047 -> 0.2047 -> 0.2047
